In [25]:
import numpy as np
from importlib import reload
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
import visualization
import timeit
import pickle
from numpy.random import multivariate_normal
from scipy.stats import multivariate_normal as mn
from sklearn.metrics import roc_curve, roc_auc_score
import topic_model
reload(topic_model)
from topic_model import TopicModel, BaseRegularizer, KLDocumentPairsRegularizer, KLWordPairsRegularizer

# 1. 
Реализовать ЕМ-алгоритм для восстановления параметров тематической модели PLSA с возможностью добавления регуляризации. Помните, что можно избежать хранения трёхмерного массива счётчиков при правильной организации процесса обучения.


# 2. 
Провести тестирование реализованного ЕМ-алгоритма на сгенерированных данных. Для этого необходимо сгенерировать столбцы Φ, Θ (например, из нормированного Гамма распределения) и по модели порождения коллекции сгенерировать коллекцию. Протестировать алгоритм на сгенерированных данных и убедиться, что правдоподобие не убывает на каждом шаге алгоритма.

In [103]:
dict_size = 100
T_number = 10
docs_number = 50

sintetic_phi = np.random.gamma(size=(dict_size, T_number), shape=2.)
sintetic_phi /= sintetic_phi.sum(axis=0)
sintetic_theta = np.random.gamma(size=(T_number, docs_number), shape=2.)
sintetic_theta /= sintetic_theta.sum(axis=0)

In [104]:
collection = np.empty((docs_number, dict_size))
for doc_number in range(docs_number):
    doc_length = np.random.randint(20, size=1, high=100, dtype=int)
    topic_distr = np.random.choice(T_number, p=sintetic_theta[:, doc_number], size=doc_length)
    word_counters = np.zeros(dict_size)
    for pos, topic in enumerate(topic_distr):
        #collection[doc_number, pos] = np.random.choice(dict_size, p=sintetic_phi[:, topic], size=1)
        word_counters[np.random.choice(dict_size, p=sintetic_phi[:, topic], size=1)] += 1
    collection[doc_number] = word_counters

In [154]:
reload(topic_model)
from topic_model import TopicModel, BaseRegularizer, KLDocumentPairsRegularizer, KLWordPairsRegularizer

In [118]:
model = TopicModel(num_topics=10)

In [119]:
model.EM_fit(data=collection, trace=True)

(array([[  3.39716406e-05,   2.59368526e-04,   1.76105690e-02,
           1.96395000e-02,   1.25933123e-02,   7.67322288e-03,
           1.59348249e-02,   4.98966498e-03,   1.60553502e-02,
           4.33300645e-03],
        [  9.07144000e-03,   5.44933877e-04,   7.89688846e-08,
           2.45516382e-03,   7.10421954e-07,   1.78780772e-02,
           2.71904304e-03,   7.38789442e-05,   5.25717195e-02,
           3.50426482e-04],
        [  2.55643935e-02,   7.58658165e-03,   1.01704266e-05,
           2.80655375e-02,   4.61086353e-06,   2.34511313e-04,
           6.78195493e-03,   5.06760289e-02,   4.18855311e-06,
           9.28141329e-07],
        [  6.08905546e-03,   2.85366107e-06,   8.03931447e-03,
           4.59686808e-04,   2.53690019e-04,   2.05583437e-03,
           1.37385979e-02,   2.65093237e-02,   1.80081764e-02,
           1.24699573e-02],
        [  5.93142221e-12,   1.01324250e-02,   1.12975297e-02,
           7.15311431e-04,   3.05579951e-06,   1.15437473e-02,
      

# 3. 
Считать обучающую выборку, сохранить датасет в памяти в формате scipy.sparse.csr_matrix.
Замечание. Для текстов проведена полная предобработка (лемматизация, выброшены стоп-слова, удале- ны лишние символы). Можно удалить из выборки слова, которые встречаются меньше чем в 5 документах, для снижения времени работы.



In [120]:
from sklearn.feature_extraction.text import CountVectorizer

In [146]:
cv = CountVectorizer(min_df=5)

In [147]:
english_data = []
with open('data_news_parallel/english_clean', 'r') as f:
    for line in f:
        english_data.append(line)
        
russian_data = []
with open('data_news_parallel/russian_clean', 'r') as f:
    for line in f:
        russian_data.append(line)
 

In [148]:
english_counters = cv.fit_transform(english_data)

In [149]:
russian_counters = cv.fit_transform(russian_data)

In [150]:
english_counters, russian_counters

(<4952x17787 sparse matrix of type '<class 'numpy.int64'>'
 	with 1305523 stored elements in Compressed Sparse Row format>,
 <4952x16118 sparse matrix of type '<class 'numpy.int64'>'
 	with 1477233 stored elements in Compressed Sparse Row format>)

# 4. 
Построить модель на русскоязычной или англоязычной части коллекции (около 50 тем), для каждой темы вывести список её топ-слов и топ-документов. Визуально оценить интерпретируемость полученных тем.



In [197]:
reload(topic_model)
from topic_model import TopicModel, BaseRegularizer, KLDocumentPairsRegularizer, KLWordPairsRegularizer

In [198]:
model = TopicModel(num_topics=50, max_iter=10)

In [ ]:
start_time = timeit.default_timer()
phi, theta, logs = model.EM_fit(data=russian_counters, trace=True)
print('time:', timeit.default_timer() - start_time)

0 batch, time: 3.03972899611108
1 batch, time: 2.755272928974591
2 batch, time: 2.650064319022931
3 batch, time: 2.7185832370305434
4 batch, time: 2.710213089012541
5 batch, time: 2.6651463570306078
6 batch, time: 2.6223725780146196
7 batch, time: 2.6281678259838372
8 batch, time: 2.6301648509688675
9 batch, time: 2.6504179789917544
10 batch, time: 2.6377989229513332
11 batch, time: 2.644797421991825
12 batch, time: 2.600712309940718
13 batch, time: 2.6389083170797676
14 batch, time: 2.5924458220833912
15 batch, time: 2.631769891944714
16 batch, time: 2.6086649139178917
17 batch, time: 2.6026501989690587
18 batch, time: 2.597847057040781
19 batch, time: 2.573850123095326
20 batch, time: 2.559239361085929
21 batch, time: 2.5824056130368263
22 batch, time: 2.606105156010017
23 batch, time: 2.5814457130618393
24 batch, time: 2.607785386033356
25 batch, time: 2.5935724929440767
26 batch, time: 2.584690217045136
27 batch, time: 2.559313341975212
28 batch, time: 2.5753982679452747
29 batch, 

/Users/Galya/Documents/studying/6th_semester/prac/task_2/prototypes/topic_model.py:228: RuntimeWarning: divide by zero encountered in log
  def get_top_words(self, k):


0 time: 136.10058961703908
0 batch, time: 2.5773683820152655
1 batch, time: 2.5748208969598636
2 batch, time: 2.627951702917926


In [ ]:
plt.plot(range(len(logs)), logs)

# 5. 
Построить двуязычную регуляризованную модель с регуляризатором на основе данных о переводах слов. Предварительно выведите формулу для производных регуляризатора. Профильтруйте приложенные сло- вари, чтобы в них остались только слова, находящиеся в выборке. Подберите коэффициент регуляризации, основываясь на правдоподобии и метриках AP@n. Визуально оцените качество полученных тем. Сделайте необходимые выводы о применимости такого подхода.




# 6. 
Построить регуляризованную модель с регуляризатором на основе данных о параллельных документах. Предварительно выведите формулу для производных регуляризатора. Используйте данные только из обу- чающей выборки. Подберите коэффициент регуляризации, основываясь на правдоподобии и метриках AP@n. Визуально оцените качество полученных тем. Сделайте необходимые выводы о применимости та- кого подхода. Попробуйте применить одновременно оба регуляризатора.